In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.model_selection import GridSearchCV


import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import data
data = pd.read_csv(r'E:\Lam Nhut Thinh\Machine Learning\data_attacker\\train.csv',index_col=0)

# Import testing data
test_data = pd.read_csv(r'E:\Lam Nhut Thinh\Machine Learning\data_attacker\\test.csv',index_col=0)

In [3]:
# model = lgb.LGBMClassifier(n_estimators=1000, learning_rate = 0.05, verbose = -1)
# model.fit(data.iloc[:,1:],data.iloc[:,0])
# importance_dt = model.feature_importances_
# features_importances_dt = pd.DataFrame({'FeatureName': data.iloc[:,1:], 
#                                         'Feature Importance': importance_dt})
# features_importances_dt['Normalized_Importance'] = features_importances_dt['Feature Importance'] / features_importances_dt['Feature Importance'].sum()
# features_importances_dt.sort_values(by=['Feature Importance'], ascending=False)

# features_importances_dt.sort_values("Feature Importance", ascending=False).iloc[:15,:].sort_values("Feature Importance").plot(figsize=(12,6),
#                                                                              x="FeatureName", 
#                                                                              y=["Normalized_Importance"], 
#                                                                              kind="barh")
# features_importances_dt
# plt.title("Feature Importance".upper())

In [4]:
# features = ['dob', 'unknown_var_17', 'unknown_var_8', 'unknown_var_9', 'unknown_var_7', 'value', 'num_date_review', 'social_friend_count', 'unknown_var_15', 'unknown_var_10', 'unknown_var_1', 'unknown_var_3', 'social_subcriber_count', 'unknown_var_12', 'unknown_var_2', 'unknown_var_14', 'unknown_var_18']

In [5]:
# data = data[['label','dob', 'unknown_var_17', 'unknown_var_8', 'unknown_var_9', 'unknown_var_7', 'value', 'num_date_review', 'social_friend_count', 'unknown_var_15', 'unknown_var_10', 'unknown_var_1', 'unknown_var_3', 'social_subcriber_count', 'unknown_var_12', 'unknown_var_2', 'unknown_var_14', 'unknown_var_18']]
# test_data = test_data[features]

In [6]:
data.head()

,label,cat_3,cat_5,mul_rate,value,cat_6,num_date_review,review_value,dob,cat_7,...,cat_1_cat_1_No,cat_2_P1,cat_2_P2,cat_2_cat_2_No,sex_FEMALE,sex_MALE,sex_sex_No,social_sex_info_female,social_sex_info_male,social_sex_info_social_sex_info_No
id,,,,,,,,,,,,,,,,,,,,,
11651,0,1.0,1.0,0.00,4357542.0,1.0,0.0,1150000.0,19821122.0,0.0,...,0,0,1,0,0,1,0,0,1,0
48491,0,1.0,1.0,0.00,4480000.0,1.0,3.0,1150000.0,19930819.0,0.0,...,0,1,0,0,0,1,0,1,0,0
42868,0,1.0,1.0,2.26,3367400.0,1.0,0.0,1150000.0,19810807.0,1.0,...,0,1,0,0,1,0,0,1,0,0
68835,1,1.0,1.0,0.00,4260000.0,1.0,82.0,1150000.0,19800416.0,0.0,...,0,1,0,0,1,0,0,0,1,0
73688,1,1.0,1.0,0.00,1490000.0,1.0,0.0,1150000.0,19900101.0,0.0,...,1,0,0,1,0,0,1,1,0,0


In [7]:
data.label.value_counts()

0    31481
1    16549
Name: label, dtype: int64

In [8]:
# Set independent variable properties
feature = data.columns[1:len(data.columns)].to_list()
target=  [data.columns[0]]

# Assign dependent and independent variables
y = data.label
X = data[feature]

print('Target: ', target)
print('Features: ', feature)

Target:  ['label']
Features:  ['cat_3', 'cat_5', 'mul_rate', 'value', 'cat_6', 'num_date_review', 'review_value', 'dob', 'cat_7', 'num_trans_last_month', 'cat_10', 'cat_11', 'unknown_var_1', 'unknown_var_2', 'unknown_var_3', 'unknown_var_4', 'unknown_var_6', 'unknown_var_7', 'unknown_var_8', 'unknown_var_9', 'unknown_var_10', 'unknown_var_11', 'unknown_var_12', 'unknown_var_13', 'unknown_var_14', 'unknown_var_15', 'unknown_var_16', 'unknown_var_17', 'unknown_var_18', 'unknown_var_19', 'unknown_var_20', 'social_friend_count', 'social_subcriber_count', 'cat_1_C1', 'cat_1_C2', 'cat_1_cat_1_No', 'cat_2_P1', 'cat_2_P2', 'cat_2_cat_2_No', 'sex_FEMALE', 'sex_MALE', 'sex_sex_No', 'social_sex_info_female', 'social_sex_info_male', 'social_sex_info_social_sex_info_No']


In [9]:
# Split the train and test
n_state = 0
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.8, test_size = 0.2, random_state=n_state)

In [10]:
def confusion_matrix_df(y_test,y_pred):
    df = pd.DataFrame(confusion_matrix(y_test,y_pred),
                      columns=['Predict 0','Predict 1'],
                      index=['Actual 0','Actual 1'])
    return df.style.background_gradient(cmap='PuBu')

In [11]:
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import MaxAbsScaler
X_train_gnb, X_test_gnb, y_train_gnb, y_test_gnb = train_test_split(X,y, train_size = 0.8, test_size = 0.2, random_state=n_state)

gnb = GaussianNB()
gnb_model = gnb.fit(X_train_gnb, y_train_gnb)
preds = gnb_model.predict(X_test_gnb)
acc_gnb = (preds == y_test_gnb).sum().astype(float) / len(preds)*100
print("Scikit-Learn's Gaussian Naive Bayes Classifier's prediction accuracy is: %3.2f" % (acc_gnb))
print(classification_report(y_test_gnb,preds))

confusion_matrix_df(y_test_gnb,preds)

Scikit-Learn's Gaussian Naive Bayes Classifier's prediction accuracy is: 39.51
              precision    recall  f1-score   support

           0       0.67      0.17      0.27      6335
           1       0.34      0.84      0.49      3271

    accuracy                           0.40      9606
   macro avg       0.50      0.50      0.38      9606
weighted avg       0.56      0.40      0.34      9606



,Predict 0,Predict 1
Actual 0,1049,5286
Actual 1,525,2746


In [12]:
my_forest = RandomForestClassifier(random_state = 0)
model_forest = my_forest.fit(X_train, y_train)
y_pred_forest = model_forest.predict(X_test)
print(classification_report(y_test, y_pred_forest))
print(f'Random Forest Accuracy: {100*round(accuracy_score(y_test, y_pred_forest),2)}%')
confusion_matrix_df(y_test,y_pred_forest)

              precision    recall  f1-score   support

           0       0.77      0.94      0.85      6335
           1       0.80      0.44      0.57      3271

    accuracy                           0.77      9606
   macro avg       0.78      0.69      0.71      9606
weighted avg       0.78      0.77      0.75      9606

Random Forest Accuracy: 77.0%


,Predict 0,Predict 1
Actual 0,5966,369
Actual 1,1819,1452


In [13]:
forest_param = {'n_estimators': [20,50,100],
                'max_depth' : [4,5,6],
                'max_features':['auto','sqrt'],
                'min_samples_leaf':[1,2,3]}
grid_forest = GridSearchCV(estimator = RandomForestClassifier(random_state = 0), 
                           param_grid=forest_param, cv= 5)
grid_forest.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=0),
             param_grid={'max_depth': [4, 5, 6],
                         'max_features': ['auto', 'sqrt'],
                         'min_samples_leaf': [1, 2, 3],
                         'n_estimators': [20, 50, 100]})

In [14]:
y_pred_grid_forest = grid_forest.predict(X_test)
print(classification_report(y_test, y_pred_grid_forest))
print(f'Random Forest Accuracy: {100*round(accuracy_score(y_test, y_pred_grid_forest),2)}%')
confusion_matrix_df(y_test,y_pred_grid_forest)

              precision    recall  f1-score   support

           0       0.66      1.00      0.79      6335
           1       1.00      0.00      0.00      3271

    accuracy                           0.66      9606
   macro avg       0.83      0.50      0.40      9606
weighted avg       0.78      0.66      0.52      9606

Random Forest Accuracy: 66.0%


,Predict 0,Predict 1
Actual 0,6335,0
Actual 1,3270,1


In [15]:
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score

my_tree = DecisionTreeClassifier(random_state = 0)
model_dectree = my_tree.fit(X_train, y_train)
y_pred_dectree = model_dectree.predict(X_test)
print(classification_report(y_test, y_pred_dectree))
print(f'Decision Tree Accuracy: {100*round(accuracy_score(y_test, y_pred_dectree),2)}%')
confusion_matrix_df(y_test,y_pred_dectree)

              precision    recall  f1-score   support

           0       0.78      0.79      0.79      6335
           1       0.58      0.58      0.58      3271

    accuracy                           0.72      9606
   macro avg       0.68      0.68      0.68      9606
weighted avg       0.72      0.72      0.72      9606

Decision Tree Accuracy: 72.0%


,Predict 0,Predict 1
Actual 0,4994,1341
Actual 1,1382,1889


In [16]:
from sklearn.model_selection import GridSearchCV

tree_param = {'max_depth':[4,5,6],
              'max_features':['auto','sqrt'],
              'min_samples_leaf':[1,2,3]}
grid_dectree = GridSearchCV(DecisionTreeClassifier(random_state = 0), tree_param, cv=5)
grid_dectree.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=0),
             param_grid={'max_depth': [4, 5, 6],
                         'max_features': ['auto', 'sqrt'],
                         'min_samples_leaf': [1, 2, 3]})

In [17]:
y_pred_grid_dectree=grid_dectree.predict(X_test)
print(classification_report(y_test, y_pred_grid_dectree))
print(f'Decision Tree Accuracy: {100*round(accuracy_score(y_test, y_pred_grid_dectree),2)}%')
confusion_matrix_df(y_test,y_pred_grid_dectree)

              precision    recall  f1-score   support

           0       0.66      1.00      0.79      6335
           1       0.49      0.01      0.02      3271

    accuracy                           0.66      9606
   macro avg       0.58      0.50      0.40      9606
weighted avg       0.60      0.66      0.53      9606

Decision Tree Accuracy: 66.0%


,Predict 0,Predict 1
Actual 0,6308,27
Actual 1,3245,26


In [18]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [19]:
from scipy.stats import pearsonr,spearmanr
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score,f1_score,recall_score,classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
lr=LogisticRegression()
model_lr = lr.fit(X_train, y_train)
y_pred_lr = model_lr.predict(X_test)
print(classification_report(y_test,y_pred_lr))
print(f'Logistic Regression Accuracy: {round(100*accuracy_score(y_test,y_pred_lr),2)}%')
confusion_matrix_df(y_test,y_pred_lr)

              precision    recall  f1-score   support

           0       0.66      1.00      0.79      6335
           1       1.00      0.00      0.00      3271

    accuracy                           0.66      9606
   macro avg       0.83      0.50      0.40      9606
weighted avg       0.78      0.66      0.52      9606

Logistic Regression Accuracy: 65.96%


,Predict 0,Predict 1
Actual 0,6335,0
Actual 1,3270,1


In [20]:
lr_tune = LogisticRegression()
lr_tune.fit(X_train,y_train)
for i in range(60,100,5):
    print(f'THRESHOLD = {i/100}')
    y_pred = np.where(lr_tune.predict_proba(X_test)[:,0] > i/100,1,0)
    print(f'Accuracy: {round(100*accuracy_score(y_test,y_pred),2)}%')
    print(classification_report(y_test,y_pred))

THRESHOLD = 0.6
Accuracy: 34.03%
              precision    recall  f1-score   support

           0       0.46      0.00      0.00      6335
           1       0.34      1.00      0.51      3271

    accuracy                           0.34      9606
   macro avg       0.40      0.50      0.26      9606
weighted avg       0.42      0.34      0.18      9606

THRESHOLD = 0.65
Accuracy: 47.01%
              precision    recall  f1-score   support

           0       0.66      0.41      0.51      6335
           1       0.34      0.58      0.43      3271

    accuracy                           0.47      9606
   macro avg       0.50      0.50      0.47      9606
weighted avg       0.55      0.47      0.48      9606

THRESHOLD = 0.7
Accuracy: 65.48%
              precision    recall  f1-score   support

           0       0.66      0.98      0.79      6335
           1       0.37      0.02      0.04      3271

    accuracy                           0.65      9606
   macro avg       0.51     

In [21]:
from sklearn.neighbors import KNeighborsClassifier
model_knn = KNeighborsClassifier()
model_knn.fit(X_train, y_train)
y_pred_knn = model_knn.predict(X_test)
print(classification_report(y_test,y_pred_knn))
print(f'KNN Accuracy: {100*round(accuracy_score(y_test, y_pred_knn),2)}%')
confusion_matrix_df(y_test,y_pred_knn)

              precision    recall  f1-score   support

           0       0.70      0.79      0.74      6335
           1       0.45      0.34      0.39      3271

    accuracy                           0.64      9606
   macro avg       0.57      0.56      0.56      9606
weighted avg       0.61      0.64      0.62      9606

KNN Accuracy: 64.0%


,Predict 0,Predict 1
Actual 0,4999,1336
Actual 1,2169,1102


In [22]:
# k_range = list(range(5, 21))
# knn_param = dict(n_neighbors = k_range,
#                  p = [1,2],
#                  leaf_size = [5,10,20,40],
#                  algorithm = ['auto', 'ball_tree', 'kd_tree', 'brute'])
  
# grid_knn = GridSearchCV(KNeighborsClassifier(), knn_param, cv=5) 
# grid_knn.fit(X_train, y_train)

In [23]:
# y_pred_grid_knn = grid_knn.predict(X_test)
# print(classification_report(y_test, y_pred_grid_knn))
# print(f'Random Forest Accuracy: {100*round(accuracy_score(y_test, y_pred_grid_knn),2)}%')
# confusion_matrix_df(y_test,y_pred_grid_knn)

In [24]:
# from sklearn import svm
# model_svm = svm.SVC()
# model_svm.fit(X_train, y_train)
# y_pred_svm = model_svm.predict(X_test)
# print(classification_report(y_test,y_pred_svm))
# print(f'SVM Accuracy: {100*round(accuracy_score(y_test, y_pred_svm),2)}%')
# confusion_matrix_df(y_test,y_pred_svm)

# **Sampling model**


In [25]:
# Split the train and test
n_state = 0
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.8, test_size = 0.2, random_state=n_state)

In [26]:
X_train

,cat_3,cat_5,mul_rate,value,cat_6,num_date_review,review_value,dob,cat_7,num_trans_last_month,...,cat_1_cat_1_No,cat_2_P1,cat_2_P2,cat_2_cat_2_No,sex_FEMALE,sex_MALE,sex_sex_No,social_sex_info_female,social_sex_info_male,social_sex_info_social_sex_info_No
id,,,,,,,,,,,,,,,,,,,,,
70401,1.0,1.0,0.0,1490000.0,1.0,0.0,1150000.0,19900101.0,0.0,5.0,...,1,0,0,1,0,0,1,0,0,1
36525,1.0,1.0,0.0,4500000.0,1.0,7.0,1150000.0,19920609.0,1.0,5.0,...,0,1,0,0,1,0,0,1,0,0
27586,1.0,1.0,0.0,8001080.0,1.0,175.0,1150000.0,19860415.0,0.0,4.0,...,0,1,0,0,1,0,0,1,0,0
11541,1.0,1.0,0.0,5397000.0,1.0,57.0,1150000.0,19930422.0,0.0,4.0,...,0,0,1,0,1,0,0,1,0,0
55488,1.0,1.0,0.0,5000000.0,1.0,0.0,1150000.0,19880315.0,0.0,5.0,...,0,0,1,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11283,1.0,1.0,0.0,3477500.0,1.0,4.0,1150000.0,19970620.0,0.0,5.0,...,0,0,1,0,0,1,0,0,0,1
48569,1.0,1.0,0.0,1490000.0,1.0,0.0,1150000.0,19900101.0,0.0,5.0,...,1,0,0,1,0,0,1,0,0,1
69684,1.0,1.0,0.0,1490000.0,1.0,0.0,1150000.0,19900101.0,0.0,5.0,...,1,0,0,1,0,0,1,0,0,1


In [27]:
y_train

id
70401    0
36525    0
27586    0
11541    1
55488    0
        ..
11283    0
48569    0
69684    1
28978    0
13068    1
Name: label, Length: 38424, dtype: int64

In [28]:
df_train = pd.concat([X_train, y_train],axis=1)

In [29]:
#  Data contains the values retained account with bank
df_majority = df_train[df_train.label==0]

#  Data contains the values closed account with bank
df_minority = df_train[df_train.label==1]

In [30]:
# The orginal shape of data
print('The orginal shape of data:')
print('class 0:', df_majority.shape)
print('class 1:', df_minority.shape)

The orginal shape of data:
class 0: (25146, 46)
class 1: (13278, 46)


In [31]:
# First, we'll import the resampling module from Scikit-Learn
from sklearn.utils import resample

# Scale up
df_minority_upsampled = resample(df_minority, 
                                 replace=True,                       # sample with replacement
                                 n_samples= df_majority.shape[0],    # to match majority class
                                 random_state=0)                    # reproducible results

# # Scale down (Worse result)
# df_majority_upsampled = resample(df_majority, 
#                                  replace=True,                       # sample with replacement
#                                  n_samples= df_minority.shape[0],    # to match majority class
#                                  random_state=45)                    # reproducible results                                

In [32]:
print('The shape of data after up-sampled:')
print('Shape of class 0:', df_majority.shape)
print('Shape of class 1:', df_minority_upsampled.shape)


# print('The shape of data after up-sampled:')
# print('Shape of class 0:', df_majority_upsampled.shape)
# print('Shape of class 1:', df_minority.shape)

The shape of data after up-sampled:
Shape of class 0: (25146, 46)
Shape of class 1: (25146, 46)


In [33]:
# Bind 2 classes together into 1 dataframe
df_upsampled = pd.concat([df_majority, df_minority_upsampled])

### Worse
# df_upsampled = pd.concat([df_majority_upsampled, df_minority])

In [34]:
df_upsampled

,cat_3,cat_5,mul_rate,value,cat_6,num_date_review,review_value,dob,cat_7,num_trans_last_month,...,cat_2_P1,cat_2_P2,cat_2_cat_2_No,sex_FEMALE,sex_MALE,sex_sex_No,social_sex_info_female,social_sex_info_male,social_sex_info_social_sex_info_No,label
id,,,,,,,,,,,,,,,,,,,,,
70401,1.0,1.0,0.0,1490000.0,1.0,0.0,1150000.0,19900101.0,0.0,5.0,...,0,0,1,0,0,1,0,0,1,0
36525,1.0,1.0,0.0,4500000.0,1.0,7.0,1150000.0,19920609.0,1.0,5.0,...,1,0,0,1,0,0,1,0,0,0
27586,1.0,1.0,0.0,8001080.0,1.0,175.0,1150000.0,19860415.0,0.0,4.0,...,1,0,0,1,0,0,1,0,0,0
55488,1.0,1.0,0.0,5000000.0,1.0,0.0,1150000.0,19880315.0,0.0,5.0,...,0,1,0,0,1,0,0,1,0,0
53845,1.0,1.0,0.0,1390000.0,1.0,0.0,1150000.0,19830621.0,0.0,33.0,...,1,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20534,1.0,1.0,0.0,3600000.0,1.0,7.0,1150000.0,19930219.0,3.0,5.0,...,1,0,0,0,1,0,1,0,0,1
89221,1.0,1.0,0.0,1490000.0,1.0,0.0,1150000.0,19900101.0,0.0,5.0,...,0,0,1,0,0,1,0,1,0,1
87579,1.0,1.0,0.0,1490000.0,1.0,0.0,1150000.0,19900101.0,0.0,5.0,...,0,0,1,0,0,1,1,0,0,1


In [35]:
# Assign dependent and independent variables
y_upsampled = df_upsampled.label
X_upsampled = df_upsampled[feature]

In [36]:
# from sklearn.naive_bayes import GaussianNB
# from sklearn.preprocessing import MaxAbsScaler

# X_train_gnb, X_test_gnb, y_train_gnb, y_test_gnb = train_test_split(X,y, train_size = 0.8, test_size = 0.2, random_state=n_state)

# gnb = GaussianNB()
# gnb_model = gnb.fit(X_train_gnb, y_train_gnb)
# preds = gnb_model.predict(X_test_gnb)
# acc_gnb = (preds == y_test_gnb).sum().astype(float) / len(preds)*100
# print("Scikit-Learn's Gaussian Naive Bayes Classifier's prediction accuracy is: %3.2f" % (acc_gnb))

In [37]:
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# X_upsampled = sc.fit_transform(X_upsampled)


In [38]:
my_forest_up = RandomForestClassifier(random_state = 0)
model_forest_up = my_forest_up.fit(X_upsampled, y_upsampled)
y_pred_forest_up = model_forest_up.predict(X_test)
print(classification_report(y_test, y_pred_forest_up))
print(f'Random Forest Accuracy: {100*round(accuracy_score(y_test, y_pred_forest_up),2)}%')
confusion_matrix_df(y_test,y_pred_forest_up)

              precision    recall  f1-score   support

           0       0.75      0.92      0.83      6335
           1       0.73      0.42      0.53      3271

    accuracy                           0.75      9606
   macro avg       0.74      0.67      0.68      9606
weighted avg       0.75      0.75      0.73      9606

Random Forest Accuracy: 75.0%


,Predict 0,Predict 1
Actual 0,5819,516
Actual 1,1892,1379


In [39]:
forest_param_up = {'n_estimators': [20,50,100],
                'max_depth' : [4,5,6],
                'max_features':['auto','sqrt'],
                'min_samples_leaf':[1,2,3]}
grid_forest_up = GridSearchCV(estimator = RandomForestClassifier(random_state = 0), 
                           param_grid=forest_param_up, cv= 5)
grid_forest_up.fit(X_upsampled, y_upsampled)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=0),
             param_grid={'max_depth': [4, 5, 6],
                         'max_features': ['auto', 'sqrt'],
                         'min_samples_leaf': [1, 2, 3],
                         'n_estimators': [20, 50, 100]})

In [40]:
y_pred_grid_forest_up = grid_forest_up.predict(X_test)
print(classification_report(y_test, y_pred_grid_forest_up))
print(f'Random Forest Accuracy: {100*round(accuracy_score(y_test, y_pred_grid_forest_up),2)}%')
confusion_matrix_df(y_test,y_pred_grid_forest_up)

              precision    recall  f1-score   support

           0       0.70      0.40      0.51      6335
           1       0.37      0.68      0.47      3271

    accuracy                           0.49      9606
   macro avg       0.53      0.54      0.49      9606
weighted avg       0.59      0.49      0.49      9606

Random Forest Accuracy: 49.0%


,Predict 0,Predict 1
Actual 0,2503,3832
Actual 1,1063,2208


In [41]:
my_tree_up = DecisionTreeClassifier(random_state = 0)
model_dectree_up = my_tree_up.fit(X_upsampled, y_upsampled)
y_pred_dectree_up = model_dectree_up.predict(X_test)
print(classification_report(y_test, y_pred_dectree_up))
print(f'Decision Tree Accuracy: {100*round(accuracy_score(y_test, y_pred_dectree_up),2)}%')
confusion_matrix_df(y_test,y_pred_dectree_up)

              precision    recall  f1-score   support

           0       0.77      0.79      0.78      6335
           1       0.58      0.55      0.57      3271

    accuracy                           0.71      9606
   macro avg       0.68      0.67      0.67      9606
weighted avg       0.71      0.71      0.71      9606

Decision Tree Accuracy: 71.0%


,Predict 0,Predict 1
Actual 0,5025,1310
Actual 1,1463,1808


In [42]:
from sklearn.model_selection import GridSearchCV

tree_param_up = {'max_depth':[4,5,6],
              'max_features':['auto','sqrt'],
              'min_samples_leaf':[1,2,3]}
grid_dectree_up = GridSearchCV(DecisionTreeClassifier(random_state = 0), tree_param_up, cv=5)
grid_dectree_up.fit(X_upsampled,y_upsampled)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=0),
             param_grid={'max_depth': [4, 5, 6],
                         'max_features': ['auto', 'sqrt'],
                         'min_samples_leaf': [1, 2, 3]})

In [43]:
y_pred_grid_dectree_up=grid_dectree_up.predict(X_test)
print(classification_report(y_test, y_pred_grid_dectree_up))
print(f'Decision Tree Accuracy: {100*round(accuracy_score(y_test, y_pred_grid_dectree_up),2)}%')
confusion_matrix_df(y_test,y_pred_grid_dectree_up)

              precision    recall  f1-score   support

           0       0.69      0.11      0.18      6335
           1       0.34      0.91      0.50      3271

    accuracy                           0.38      9606
   macro avg       0.52      0.51      0.34      9606
weighted avg       0.57      0.38      0.29      9606

Decision Tree Accuracy: 38.0%


,Predict 0,Predict 1
Actual 0,666,5669
Actual 1,295,2976


# **Predict on test datasets**

In [44]:
y_pred = model_dectree.predict(test_data)

y_pred_upsampled = grid_dectree_up.predict(test_data)

In [45]:
res = pd.DataFrame(y_pred,columns = ['actual'])
res_usampled = pd.DataFrame(y_pred_upsampled,columns = ['actual'])

In [46]:
res.value_counts()

actual
0         3800
1         1200
dtype: int64

In [47]:
res_usampled.value_counts()

actual
1         4008
0          992
dtype: int64

In [48]:
res = res.set_index('actual')
res_usampled = res_usampled.set_index('actual')


In [49]:
# res.to_excel('pred8.xlsx')

In [50]:
res_list = {"Model":["Decision Tree Classifier", "Logistic Regression", "Random Forest Classifier", "KNN Classifier", "Support Vector Machine", "Naive Bayes"],
           "Accuracy":[0.38,0.35,0.49,0.63,0.66,0.40],
           "F1 Score":[0.29,0.22,0.49,0.62,0.52,0.34],
           "Precision":[0.34,0.34,0.37,0.45,0,0.34],
           "Recall":[0.91,0.96,0.68,0.34,0,0.84]}
res_list = pd.DataFrame(res_list)
res_list = res_list.set_index('Model')
res_list

,Accuracy,F1 Score,Precision,Recall
Model,,,,
Decision Tree Classifier,0.38,0.29,0.34,0.91
Logistic Regression,0.35,0.22,0.34,0.96
Random Forest Classifier,0.49,0.49,0.37,0.68
KNN Classifier,0.63,0.62,0.45,0.34
Support Vector Machine,0.66,0.52,0.00,0.00
Naive Bayes,0.40,0.34,0.34,0.84
